In [1]:
# Kala Project 1 Notebook

In [27]:
# Dependencies
import pandas as pd
from prophet import Prophet
import datetime as dt
import numpy as np
%matplotlib inline


In [46]:
# Read the avg-household-size data into a DataFrame
df_household_size = pd.read_csv("Resources/avg-household-size.csv")
#    index_col='geography')
df_household_size.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             3220 non-null   int64  
 1   statefips         3220 non-null   int64  
 2   countyfips        3220 non-null   int64  
 3   avghouseholdsize  3220 non-null   float64
 4   geography         3220 non-null   object 
dtypes: float64(1), int64(3), object(1)
memory usage: 125.9+ KB


In [47]:
# Dislay the df
df_household_size

,index,statefips,countyfips,avghouseholdsize,geography
0,0,2,13,2.43,"Aleutians East Borough, Alaska"
1,1,2,16,3.59,"Aleutians West Census Area, Alaska"
2,2,2,20,2.77,"Anchorage Municipality, Alaska"
3,3,2,50,3.86,"Bethel Census Area, Alaska"
4,4,2,60,2.50,"Bristol Bay Borough, Alaska"
...,...,...,...,...,...
3215,3215,56,37,2.64,"Sweetwater County, Wyoming"
3216,3216,56,39,2.63,"Teton County, Wyoming"
3217,3217,56,41,2.76,"Uinta County, Wyoming"
3218,3218,56,43,2.34,"Washakie County, Wyoming"


In [49]:
#
df_household_size[['County','State']] = df_household_size['geography'].str.split(',', expand = True)

df_household_size['County'] = df_household_size['County'].str.strip()
df_household_size['State'] = df_household_size['State'].str.strip()

df_household_size.info()
display(df_household_size)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             3220 non-null   int64  
 1   statefips         3220 non-null   int64  
 2   countyfips        3220 non-null   int64  
 3   avghouseholdsize  3220 non-null   float64
 4   geography         3220 non-null   object 
 5   County            3220 non-null   object 
 6   State             3220 non-null   object 
dtypes: float64(1), int64(3), object(3)
memory usage: 176.2+ KB


,index,statefips,countyfips,avghouseholdsize,geography,County,State
0,0,2,13,2.43,"Aleutians East Borough, Alaska",Aleutians East Borough,Alaska
1,1,2,16,3.59,"Aleutians West Census Area, Alaska",Aleutians West Census Area,Alaska
2,2,2,20,2.77,"Anchorage Municipality, Alaska",Anchorage Municipality,Alaska
3,3,2,50,3.86,"Bethel Census Area, Alaska",Bethel Census Area,Alaska
4,4,2,60,2.50,"Bristol Bay Borough, Alaska",Bristol Bay Borough,Alaska
...,...,...,...,...,...,...,...
3215,3215,56,37,2.64,"Sweetwater County, Wyoming",Sweetwater County,Wyoming
3216,3216,56,39,2.63,"Teton County, Wyoming",Teton County,Wyoming
3217,3217,56,41,2.76,"Uinta County, Wyoming",Uinta County,Wyoming
3218,3218,56,43,2.34,"Washakie County, Wyoming",Washakie County,Wyoming


In [55]:
# Search for Alaska
search_state = df_household_size[df_household_size['State'] == 'Alaska']
display(search_state)

,index,statefips,countyfips,avghouseholdsize,geography,County,State
0,0,2,13,2.43,"Aleutians East Borough, Alaska",Aleutians East Borough,Alaska
1,1,2,16,3.59,"Aleutians West Census Area, Alaska",Aleutians West Census Area,Alaska
2,2,2,20,2.77,"Anchorage Municipality, Alaska",Anchorage Municipality,Alaska
3,3,2,50,3.86,"Bethel Census Area, Alaska",Bethel Census Area,Alaska
4,4,2,60,2.50,"Bristol Bay Borough, Alaska",Bristol Bay Borough,Alaska
5,5,2,68,2.34,"Denali Borough, Alaska",Denali Borough,Alaska
6,6,2,70,3.56,"Dillingham Census Area, Alaska",Dillingham Census Area,Alaska
7,7,2,90,2.69,"Fairbanks North Star Borough, Alaska",Fairbanks North Star Borough,Alaska
8,8,2,100,2.12,"Haines Borough, Alaska",Haines Borough,Alaska
9,9,2,105,2.26,"Hoonah-Angoon Census Area, Alaska",Hoonah-Angoon Census Area,Alaska


In [34]:
# Read the cancer data into a DataFrame
df_cancer = pd.read_csv("Resources/cancer_reg.csv")
#    index_col='geography')
df_cancer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047 entries, 0 to 3046
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    3047 non-null   int64  
 1   avganncount              3047 non-null   float64
 2   avgdeathsperyear         3047 non-null   int64  
 3   target_deathrate         3047 non-null   float64
 4   incidencerate            3047 non-null   float64
 5   medincome                3047 non-null   int64  
 6   popest2015               3047 non-null   int64  
 7   povertypercent           3047 non-null   float64
 8   studypercap              3047 non-null   float64
 9   binnedinc                3047 non-null   object 
 10  medianage                3047 non-null   float64
 11  medianagemale            3047 non-null   float64
 12  medianagefemale          3047 non-null   float64
 13  geography                3047 non-null   object 
 14  percentmarried          

In [35]:
df_cancer

,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
0,0,1397.000000,469,164.9,489.800000,61898,260131,11.2,499.748204,"(61494.5, 125635]",...,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
1,1,173.000000,70,161.3,411.600000,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
2,2,102.000000,50,174.7,349.700000,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",...,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488
3,3,427.000000,202,194.8,430.400000,44243,75882,17.1,342.637253,"(42724.4, 45201]",...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
4,4,57.000000,26,144.4,350.100000,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042,3042,1962.667684,15,149.6,453.549422,46961,6343,12.4,0.000000,"(45201, 48021.6]",...,54.9,44.6,31.7,13.2,90.280811,3.837754,0.327613,1.700468,51.063830,7.773512
3043,3043,1962.667684,43,150.1,453.549422,48609,37118,18.8,377.175494,"(48021.6, 51046.4]",...,53.3,48.6,28.8,17.7,75.706245,2.326771,4.044920,14.130288,52.007937,8.186470
3044,3044,1962.667684,46,153.9,453.549422,51144,34536,15.0,1968.959926,"(51046.4, 54545.6]",...,52.6,47.8,26.6,16.8,87.961629,2.313188,1.316472,5.680705,55.153949,7.809192
3045,3045,1962.667684,52,175.0,453.549422,50745,25609,13.3,0.000000,"(48021.6, 51046.4]",...,56.3,49.6,29.5,14.0,92.905681,1.176562,0.244632,2.131790,58.484232,7.582938


In [50]:
#
df_cancer[['County','State']] = df_cancer['geography'].str.split(',', expand = True)

df_cancer['County'] = df_cancer['County'].str.strip()
df_cancer['State'] = df_cancer['State'].str.strip()

df_cancer.info()
display(df_cancer)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3047 entries, 0 to 3046
Data columns (total 36 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index                    3047 non-null   int64  
 1   avganncount              3047 non-null   float64
 2   avgdeathsperyear         3047 non-null   int64  
 3   target_deathrate         3047 non-null   float64
 4   incidencerate            3047 non-null   float64
 5   medincome                3047 non-null   int64  
 6   popest2015               3047 non-null   int64  
 7   povertypercent           3047 non-null   float64
 8   studypercap              3047 non-null   float64
 9   binnedinc                3047 non-null   object 
 10  medianage                3047 non-null   float64
 11  medianagemale            3047 non-null   float64
 12  medianagefemale          3047 non-null   float64
 13  geography                3047 non-null   object 
 14  percentmarried          

,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate,County,State
0,0,1397.000000,469,164.9,489.800000,61898,260131,11.2,499.748204,"(61494.5, 125635]",...,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831,Kitsap County,Washington
1,1,173.000000,70,161.3,411.600000,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",...,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096,Kittitas County,Washington
2,2,102.000000,50,174.7,349.700000,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",...,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488,Klickitat County,Washington
3,3,427.000000,202,194.8,430.400000,44243,75882,17.1,342.637253,"(42724.4, 45201]",...,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841,Lewis County,Washington
4,4,57.000000,26,144.4,350.100000,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",...,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657,Lincoln County,Washington
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3042,3042,1962.667684,15,149.6,453.549422,46961,6343,12.4,0.000000,"(45201, 48021.6]",...,31.7,13.2,90.280811,3.837754,0.327613,1.700468,51.063830,7.773512,Ellsworth County,Kansas
3043,3043,1962.667684,43,150.1,453.549422,48609,37118,18.8,377.175494,"(48021.6, 51046.4]",...,28.8,17.7,75.706245,2.326771,4.044920,14.130288,52.007937,8.186470,Finney County,Kansas
3044,3044,1962.667684,46,153.9,453.549422,51144,34536,15.0,1968.959926,"(51046.4, 54545.6]",...,26.6,16.8,87.961629,2.313188,1.316472,5.680705,55.153949,7.809192,Ford County,Kansas
3045,3045,1962.667684,52,175.0,453.549422,50745,25609,13.3,0.000000,"(48021.6, 51046.4]",...,29.5,14.0,92.905681,1.176562,0.244632,2.131790,58.484232,7.582938,Franklin County,Kansas


In [59]:
# Search for Alaska
search_state = df_cancer[df_cancer['State'] == 'Alaska']
display(search_state.head(20))


,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate,County,State
2713,2713,8.0,3,203.3,201.3,68387,5702,9.9,0.000000,"(61494.5, 125635]",...,11.2,6.2,31.562280,4.732583,37.156932,5.295567,42.882250,4.391052,Aleutians West Census Area,Alaska
2714,2714,1085.0,352,167.5,453.3,75200,298695,10.0,565.794540,"(61494.5, 125635]",...,24.5,12.9,64.944652,5.854427,8.800530,1.747201,48.361303,5.994294,Anchorage Municipality,Alaska
2715,2715,37.0,20,218.8,338.2,45808,17946,23.8,0.000000,"(45201, 48021.6]",...,47.3,36.1,12.016202,0.798830,1.175743,0.343159,45.521390,9.156280,Bethel Census Area,Alaska
2717,2717,16.0,7,216.6,409.2,51082,4997,17.5,0.000000,"(51046.4, 54545.6]",...,41.2,30.2,18.718618,0.341434,1.446074,0.482025,46.478873,11.801242,Dillingham Census Area,Alaska
2718,2718,307.0,105,177.2,424.4,67801,99631,9.5,270.999990,"(61494.5, 125635]",...,22.2,10.4,76.649115,4.352841,2.628755,0.891630,50.354112,7.382578,Fairbanks North Star Borough,Alaska
2719,2719,13.0,4,126.3,413.3,67260,2534,9.8,0.000000,"(61494.5, 125635]",...,24.4,9.3,80.546875,0.039062,3.007812,0.507812,47.789116,5.374280,Haines Borough,Alaska
2720,2720,119.0,41,166.3,390.1,80835,32756,7.9,0.000000,"(61494.5, 125635]",...,21.6,8.7,68.995727,1.269558,6.811964,1.186561,49.380882,5.601633,Juneau City and Borough,Alaska
2721,2721,266.0,96,168.1,441.4,62532,58059,11.5,0.000000,"(61494.5, 125635]",...,29.2,14.5,84.086262,0.634382,1.279251,0.559235,51.021643,5.680094,Kenai Peninsula Borough,Alaska
2722,2722,63.0,25,205.0,477.3,57776,13709,12.4,0.000000,"(54545.6, 61494.5]",...,28.3,14.4,67.742171,0.423389,7.307103,0.656982,46.667932,4.936668,Ketchikan Gateway Borough,Alaska
2723,2723,43.0,15,181.9,416.6,67972,13889,8.3,0.000000,"(61494.5, 125635]",...,23.6,14.6,54.025621,0.472340,21.276748,2.798254,55.429321,6.768559,Kodiak Island Borough,Alaska


In [53]:
df_combined = pd.merge(df_household_size, df_cancer, on=["County", "State"], how="outer")
# df_combined = pd.merge(df_household_size, df_cancer, left_on='County', right_on='State', how='inner')  Can't do that

df_combined.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3220 entries, 0 to 3219
Data columns (total 41 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index_x                  3220 non-null   int64  
 1   statefips                3220 non-null   int64  
 2   countyfips               3220 non-null   int64  
 3   avghouseholdsize         3220 non-null   float64
 4   geography_x              3220 non-null   object 
 5   County                   3220 non-null   object 
 6   State                    3220 non-null   object 
 7   index_y                  3047 non-null   float64
 8   avganncount              3047 non-null   float64
 9   avgdeathsperyear         3047 non-null   float64
 10  target_deathrate         3047 non-null   float64
 11  incidencerate            3047 non-null   float64
 12  medincome                3047 non-null   float64
 13  popest2015               3047 non-null   float64
 14  povertypercent          

In [61]:
display(df_combined)

,index_x,statefips,countyfips,avghouseholdsize,geography_x,County,State,index_y,avganncount,avgdeathsperyear,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
0,0,2,13,2.43,"Aleutians East Borough, Alaska",Aleutians East Borough,Alaska,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2,16,3.59,"Aleutians West Census Area, Alaska",Aleutians West Census Area,Alaska,2713.0,8.0,3.0,...,60.4,58.3,11.2,6.2,31.562280,4.732583,37.156932,5.295567,42.882250,4.391052
2,2,2,20,2.77,"Anchorage Municipality, Alaska",Anchorage Municipality,Alaska,2714.0,1085.0,352.0,...,NaN,45.9,24.5,12.9,64.944652,5.854427,8.800530,1.747201,48.361303,5.994294
3,3,2,50,3.86,"Bethel Census Area, Alaska",Bethel Census Area,Alaska,2715.0,37.0,20.0,...,25.8,25.1,47.3,36.1,12.016202,0.798830,1.175743,0.343159,45.521390,9.156280
4,4,2,60,2.50,"Bristol Bay Borough, Alaska",Bristol Bay Borough,Alaska,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3215,3215,56,37,2.64,"Sweetwater County, Wyoming",Sweetwater County,Wyoming,1189.0,155.0,49.0,...,62.7,59.0,21.9,11.3,92.133476,0.866613,0.857679,1.684088,54.385755,8.929776
3216,3216,56,39,2.63,"Teton County, Wyoming",Teton County,Wyoming,1190.0,83.0,23.0,...,67.2,46.1,14.4,5.5,93.160325,0.322711,1.900408,2.783380,50.580188,1.745232
3217,3217,56,41,2.76,"Uinta County, Wyoming",Uinta County,Wyoming,1191.0,68.0,24.0,...,NaN,54.1,24.4,13.0,94.233158,0.195891,0.119446,1.538462,57.451346,6.918372
3218,3218,56,43,2.34,"Washakie County, Wyoming",Washakie County,Wyoming,1192.0,50.0,21.0,...,49.3,43.8,32.2,13.9,91.238095,0.678571,0.166667,4.142857,51.708428,5.370370


In [62]:
# Display the percentage of missing values for each column
missing_data = df_combined.isnull().mean() * 100
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

missing_data

pctsomecol18_24            76.335404
pctprivatecoveragealone    24.285714
pctemployed16_over         10.093168
pctempprivcoverage          5.372671
pcths25_over                5.372671
pctbachdeg25_over           5.372671
pctunemployed16_over        5.372671
pctprivatecoverage          5.372671
pctpubliccoverage           5.372671
avganncount                 5.372671
pctpubliccoveragealone      5.372671
pctwhite                    5.372671
pctblack                    5.372671
pctasian                    5.372671
pctotherrace                5.372671
pctmarriedhouseholds        5.372671
pctbachdeg18_24             5.372671
index_y                     5.372671
pcths18_24                  5.372671
pctnohs18_24                5.372671
percentmarried              5.372671
geography_y                 5.372671
medianagefemale             5.372671
medianagemale               5.372671
medianage                   5.372671
binnedinc                   5.372671
studypercap                 5.372671
p

In [14]:
# Analysis of Mission Data
# pctsomecol18_24 = 76.3% missing data
# pcyprivatecoveragealone = 24.3% missing data
# the other coumns ~ 5.37% missing data

# Do we hadle missing data by removing?
relevant_columns = ['avghouseholdsize', 'incidencerate', 'target_deathrate']
# type(relevant_columns)

df_relevant = df_combined[relevant_columns]
display(df_relevant)

# Drop rows with missing values in the relevant columns
df_relevant_cleaned = df_relevant.dropna()
display(df_relevant_cleaned)

,avghouseholdsize,incidencerate,target_deathrate
geography,,,
"Aleutians East Borough, Alaska",2.43,NaN,NaN
"Aleutians West Census Area, Alaska",3.59,201.3,203.3
"Anchorage Municipality, Alaska",2.77,453.3,167.5
"Bethel Census Area, Alaska",3.86,338.2,218.8
"Bristol Bay Borough, Alaska",2.50,NaN,NaN
...,...,...,...
"Sweetwater County, Wyoming",2.64,403.5,141.1
"Teton County, Wyoming",2.63,402.7,136.9
"Uinta County, Wyoming",2.76,359.3,132.8


,avghouseholdsize,incidencerate,target_deathrate
geography,,,
"Aleutians West Census Area, Alaska",3.59,201.3,203.3
"Anchorage Municipality, Alaska",2.77,453.3,167.5
"Bethel Census Area, Alaska",3.86,338.2,218.8
"Dillingham Census Area, Alaska",3.56,409.2,216.6
"Fairbanks North Star Borough, Alaska",2.69,424.4,177.2
...,...,...,...
"Sweetwater County, Wyoming",2.64,403.5,141.1
"Teton County, Wyoming",2.63,402.7,136.9
"Uinta County, Wyoming",2.76,359.3,132.8


In [16]:
# Calculate the correlation matrix
correlation_matrix = df_relevant_cleaned.corr()
display(correlation_matrix)

,avghouseholdsize,incidencerate,target_deathrate
avghouseholdsize,1.000000,-0.146103,-0.037396
incidencerate,-0.146103,1.000000,0.449432
target_deathrate,-0.037396,0.449432,1.000000


In [26]:
# correlation_pctwhite_pctblack_direct = df_combined['pctwhite'].corr(df_combined['pctblack'])

# Extract the correlations between the specified columns
correlation_pctwhite_pctblack_direct = df_combined['pctwhite'].corr(df_combined['pctblack'])
correlation_pctwhite_pctasian_direct = df_combined['pctwhite'].corr(df_combined['pctasian'])
correlation_pctblack_pctasian_direct = df_combined['pctblack'].corr(df_combined['pctasian'])

print("\nDirect correlation between pctwhite and pctblack:")
print(correlation_pctwhite_pctblack_direct)

print("\nDirect correlation between pctwhite and pctasian:")
print(correlation_pctwhite_pctasian_direct)

print("\nDirect correlation between pctblack and pctasian:")
print(correlation_pctblack_pctasian_direct)


Direct correlation between pctwhite and pctblack:
-0.8284588517024312

Direct correlation between pctwhite and pctasian:
-0.265676411146566

Direct correlation between pctblack and pctasian:
0.016583406881569375


In [ ]:
correlations = {
    'pctwhite_pctblack': correlation_matrix.loc['pctwhite', 'pctblack'],
    'pctwhite_pctasian': correlation_matrix.loc['pctwhite', 'pctasian'],
    'pctwhite_pctotherrace': correlation_matrix.loc['pctwhite', 'pctotherrace'],
    'pctwhite_popest2015': correlation_matrix.loc['pctwhite', 'popest2015'],
    'pctblack_pctasian': correlation_matrix.loc['pctblack', 'pctasian'],
    'pctblack_pctotherrace': correlation_matrix.loc['pctblack', 'pctotherrace'],
    'pctblack_popest2015': correlation_matrix.loc['pctblack', 'popest2015'],
    'pctasian_pctotherrace': correlation_matrix.loc['pctasian', 'pctotherrace'],
    'pctasian_popest2015': correlation_matrix.loc['pctasian', 'popest2015'],
    'pctotherrace_popest2015': correlation_matrix.loc['pctotherrace', 'popest2015']
}

# Print specific correlations
for key, value in correlations.items():
    print(f"Correlation between {key.replace('_', ' and ')}: {value}")



In [ ]:
correlations = {
    'pctwhite_pctblack': correlation_matrix.loc['pctwhite', 'pctblack'],
    'pctwhite_pctasian': correlation_matrix.loc['pctwhite', 'pctasian'],
    'pctwhite_pctotherrace': correlation_matrix.loc['pctwhite', 'pctotherrace'],
    'pctwhite_popest2015': correlation_matrix.loc['pctwhite', 'popest2015'],
    'pctblack_pctasian': correlation_matrix.loc['pctblack', 'pctasian'],
    'pctblack_pctotherrace': correlation_matrix.loc['pctblack', 'pctotherrace'],
    'pctblack_popest2015': correlation_matrix.loc['pctblack', 'popest2015'],
    'pctasian_pctotherrace': correlation_matrix.loc['pctasian', 'pctotherrace'],
    'pctasian_popest2015': correlation_matrix.loc['pctasian', 'popest2015'],
    'pctotherrace_popest2015': correlation_matrix.loc['pctotherrace', 'popest2015']
}

# Print specific correlations
for key, value in correlations.items():
    print(f"Correlation between {key.replace('_', ' and ')}: {value}")
